## Reading Data from S3 and Building Grain Pipelines

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/grain/blob/main/docs/tutorials/dataset_load_from_s3_tutorial.ipynb)

This document outlines how to read data from an Amazon S3 bucket and construct a Grain pipeline. We will leverage [S3 Mountpoint](https://docs.aws.amazon.com/AmazonS3/latest/userguide/mountpoint.html), a service provided by AWS. S3 Mountpoint enables you to mount your S3 bucket as a local file system, allowing you to access and read data as if it were stored locally.

### Install Mountpoint for Amazon S3

In [ ]:
!wget https://s3.amazonaws.com/mountpoint-s3-release/latest/x86_64/mount-s3.deb

In [ ]:
!sudo apt-get install -y ./mount-s3.deb

### Configure AWS credentials

In [ ]:
!pip install aws configure
!pip install awscli

In [ ]:
!aws configure

### Mount your S3 bucket to your local filepath

In [ ]:
!mount-s3 <your-s3-bucket> /path/to/mount/files

### Install Grain and other dependencies

In [ ]:
!pip install grain
!pip install array_record

### Write temp ArrayRecord files to the bucket

In [ ]:
from array_record.python import array_record_module

digits = [b"1", b"2", b"3", b"4", b"5"]

writer = array_record_module.ArrayRecordWriter("/path/to/mount/files/data.array_record")
for i in digits:
  writer.write(i)
writer.close()

### Read ArrayRecord files using Grain

In [ ]:
import grain
from pprint import pprint

source =  grain.sources.ArrayRecordDataSource(paths="/path/to/mount/files/data.array_record")

dataset = (
    grain.MapDataset.source(source)
    .shuffle(seed=10)  # Shuffles globally.
    .batch(batch_size=2)  # Batches consecutive elements.
)

pprint(list(dataset))